<a href="https://colab.research.google.com/github/jiheddachraoui/occupancy_grid_generator/blob/main/OG_generator_saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARATION

In [ ]:
import os
import sys
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from sklearn.linear_model import RANSACRegressor
from scipy import stats
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
datadir = '/content/drive/MyDrive/s_data/'
scenario='scenarios/scenario7/'



for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    item_path = os.path.join(os.path.join(datadir,scenario), item)
    # Check if the item is a directory
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        # Print the path of the subdirectory
        print(item_path)



/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra
/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_3 vehicle.citroen.c3
/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_1 vehicle.tesla.model3
/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2


In [ ]:
vehicles={}
for idx, item in enumerate(os.listdir(os.path.join(datadir, scenario))):
    # Construct the full path to the item
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        folder_path = os.path.join(os.path.join(datadir,scenario), item)
        lidar_path=folder_path+'/Lidar'
        img_path=folder_path+'/rgb'
        save_path=folder_path+'/occupancy_grid'
        save_path1=folder_path+'/lidar_nonroad'
        lidar_matrices_path=folder_path+ "/pos_matrices/lidar_2_world"
        camera_matrices_path=folder_path+"/pos_matrices/world_2_camera"
        camera_intrinsic_path= folder_path+"/pos_matrices/K.npy"

        if not os.path.exists(save_path):
            os.makedirs(save_path)
        num_files = len([f for f in os.listdir(lidar_path) if os.path.isfile(os.path.join(lidar_path, f))])
        print('Number of files in {} :'.format(lidar_path), num_files)
        vehicles[idx] = {
            'lidar_path': lidar_path,
            'img_path': img_path,
            'lidar_matrices_path': lidar_matrices_path,
            'camera_matrices_path': camera_matrices_path,
            'camera_intrinsic_path' :camera_intrinsic_path,
            'num_files': num_files,
            'save_path': save_path,
            'save_path1':save_path1,
        }

Number of files in /content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/Lidar : 25
Number of files in /content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_3 vehicle.citroen.c3/Lidar : 25
Number of files in /content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_1 vehicle.tesla.model3/Lidar : 25
Number of files in /content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/Lidar : 25


In [ ]:
vehicles

{0: {'lidar_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/Lidar',
  'img_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/rgb',
  'lidar_matrices_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/pos_matrices/lidar_2_world',
  'camera_matrices_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/pos_matrices/world_2_camera',
  'camera_intrinsic_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/pos_matrices/K.npy',
  'num_files': 25,
  'save_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/occupancy_grid',
  'save_path1': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/lidar_nonroad'},
 1: {'lidar_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_3 vehicle.citroen.c3/Lidar',
  'img_path': '/content/drive/

In [ ]:
ALPHA = 1
BHETA = 1*np.pi/180
RESOLUTION = 0.1
MAX_RANGE = 50
MAP_WIDTH = 100
SPHERICAL2CARTESIAN_BIAS = 6
### Some derived parameters
# OOR_MASK = A mask to filter out measurements that are out of MAX_RANGE
MAP_SIZE_X = int(MAP_WIDTH/RESOLUTION)
MAP_SIZE_Y = int(MAP_WIDTH/RESOLUTION)
xarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
yarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
MAP_XX, MAP_YY = np.meshgrid(xarr, -yarr)
rgrid = np.sqrt(np.add(np.square(MAP_XX),np.square(MAP_YY)))
OOR_MASK = rgrid >= MAX_RANGE

In [ ]:
def transform_coordinate(lidar_points,l2w_matrix,w2c_matrix):
  inp = lidar_points.copy()
  ones_column = np.ones((inp.shape[0], 1))
  inp=np.concatenate((inp, ones_column), axis=1)



  # Transform the points from lidar space to world space.
  world_points = np.dot(l2w_matrix, inp.T)
  inp = np.dot(w2c_matrix, world_points).T


  return inp[:,:3]

In [ ]:
def project_lidar2cam(lidar_in_cam,camera_intrinsic,img_raw_size):
  ### Filter out data behind the cam

  lidar_in_cam = np.concatenate((lidar_in_cam,np.ones((lidar_in_cam.shape[0],1))),axis=1)

  lidar_in_cam = lidar_in_cam[lidar_in_cam[:,0]>0]

  ### Project points to the image
  point_in_camera_coords=np.array([
                lidar_in_cam[:,1],
                lidar_in_cam[:,2] * -1 ,
                lidar_in_cam[:,0]])
  points_2d = np.matmul(camera_intrinsic,point_in_camera_coords)

  # normalize the x, y values by the 3rd value.
  points_2d = np.array([
                points_2d[0, :] / points_2d[2, :],
                points_2d[1, :] / points_2d[2, :],
                points_2d[2, :]])
  points_2d = points_2d.T
  points_2d = points_2d.astype(int)

  ### Filter out points that are outside image frame
  maskH = np.logical_and(points_2d[:,0]>=0,points_2d[:,0]<img_raw_size[1])
  maskV = np.logical_and(points_2d[:,1]>=0,points_2d[:,1]<img_raw_size[0])
  mask = np.logical_and(maskH,maskV)

  points_2d = points_2d[mask,:]
  lidar_in_cam = lidar_in_cam[mask,:]

  return points_2d,lidar_in_cam[:,:3],mask

In [ ]:
def crop_data(img_in,lidar_2d_in,lidar_in_cam_in,rh,rw):
  lidar_2d = lidar_2d_in.copy()
  lidar_in_cam = lidar_in_cam_in.copy()
  img = img_in.copy()

  ### Crop the image
  dim_ori = np.array(img.shape)
  cent = (dim_ori/2).astype(int)
  if dim_ori[0]/dim_ori[1] == rh/rw:
      crop_img = img

  # If Height <= Width
  elif dim_ori[0] <= dim_ori[1]:
      cH2 = dim_ori[0]
      cW2 = cH2*rw/rh
      cW = int(cW2/2)
      crop_img = img[:,cent[1]-cW:cent[1]+cW+1]

  # If Height > Width
  else:
      cW2 = dim_ori[1]
      cH2 = cW2*rh/rw
      cH = int(cH2/2)
      crop_img = img[cent[0]-cH:cent[0]+cH+1,:]

  ### Filter out LiDAR points outside cropped image
  cW = cW2/2
  cH = cH2/2
  centH = cent[0]
  centW = cent[1]
  maskH = np.logical_and(lidar_2d[:,1]>=centH-cH,lidar_2d[:,1]<=centH+cH)
  maskW = np.logical_and(lidar_2d[:,0]>=centW-cW,lidar_2d[:,0]<=centW+cW)
  mask = np.logical_and(maskH,maskW)
  lidar_2d = lidar_2d[mask,:]
  lidar_in_cam = lidar_in_cam[mask,:]
  cent = np.array((centW-cW,centH-cH,0)).reshape((1,3))
  lidar_2d = lidar_2d - cent

  return crop_img, lidar_2d.astype(int), lidar_in_cam


In [ ]:
def filter_road_points(lidar_in, is_road, threshold=0.5):
  y_true = lidar_in[:,1]
  y_pred = np.zeros_like(y_true)

  y_pred = y_true * is_road  # set road points' predicted height to true height
  delta_y = np.absolute(y_true-y_pred).flatten()
  is_not_road = delta_y > threshold
  lidar_out = lidar_in[is_not_road,:].copy()
  return lidar_out

In [ ]:
def generate_measurement_ogm(lidar_in,ogm_shape):
  ### Calculate the position of LiDAR points in spherical coordinate
  rphi_meas = np.zeros((lidar_in.shape[0],2))
  rphi_meas[:,1] = np.sqrt(np.add(np.square(lidar_in[:,0]),np.square(lidar_in[:,1])))/ALPHA
  rphi_meas[:,0] = (np.arctan2(lidar_in[:,1],lidar_in[:,0])+np.pi)/BHETA
  rphi_meas = np.unique(rphi_meas.astype(int),axis=0)
  rphi_meas = rphi_meas[rphi_meas[:,1]<int(MAX_RANGE/ALPHA),:]
  rphi_meas = rphi_meas[rphi_meas[:,0]<int(2*np.pi/BHETA),:]

  ### Initiate and fill the spherical scan grid
  sg_ang_bin = int(2*np.pi/BHETA)
  sg_rng_bin = int(MAX_RANGE/ALPHA)
  # Initiation (Condition 3)
  scan_grid = np.ones((sg_ang_bin,sg_rng_bin))*0.5
  # Condition 1
  scan_grid[tuple(rphi_meas.T)] = 0.7
  # Condition 2
  for ang in range(sg_ang_bin):
    ang_arr = rphi_meas[rphi_meas[:,0]==ang,1]
    if len(ang_arr) == 0:
      scan_grid[ang,:] = 0.3
    else:
      min_r = np.min(ang_arr)
      scan_grid[ang,:min_r] = 0.3

  ### Convert the spherical scan grid to the cartesian one
  ogm_sz = (ogm_shape[1],ogm_shape[0])
  ogm_cen = (int(ogm_shape[1]/2),int(ogm_shape[0]/2))
  radius = (MAX_RANGE/RESOLUTION) + SPHERICAL2CARTESIAN_BIAS
  ogm_step = cv2.warpPolar(scan_grid,ogm_sz,ogm_cen,radius,cv2.WARP_INVERSE_MAP)
  ogm_step[OOR_MASK] = 0.5
  ogm_step = cv2.rotate(ogm_step, cv2.ROTATE_90_CLOCKWISE)
  #print(ogm_step)
  return ogm_step

In [ ]:
### Function to process the image with DeepLabv3+
def process_images(img_in, sess, target_size=513, probability_threshold=0.5):
  INPUT_TENSOR_NAME = 'ImageTensor:0'
  PROB_TENSOR_NAME = 'SemanticProbabilities:0'
  INPUT_SIZE = target_size

  image = img_in.copy()
  sz = image.shape
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Resize input image to target size
  if INPUT_SIZE == 0:
    resized_image = image.copy()
  else:
    resized_image = cv2.resize(image,(INPUT_SIZE,INPUT_SIZE))

  # Run deep learning inference
  batch_seg_map = sess.run(
      PROB_TENSOR_NAME,
      feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
  seg_map = (batch_seg_map[0][:,:,1]*255).astype(int)
  prob = np.array(seg_map, dtype=np.uint8)
  prob = cv2.resize(prob,(sz[1],sz[0]))

  # Create the prediction
  pred = prob.copy()
  msk_bin = prob >= (probability_threshold*255)
  pred[msk_bin] = 1
  pred[np.logical_not(msk_bin)] = 0

  # Ignore regions that are separated from the main road
  # This can reduce the amount of false detection
  _,segm_reg = cv2.connectedComponents(pred)
  segm_reg = segm_reg.astype(float)
  segm_reg[segm_reg==0] = np.nan
  modes,_ = stats.mode(segm_reg.flatten(),axis=None,nan_policy="omit")
  mode = modes[0]
  pred[segm_reg!=mode] = 0

  return prob,(pred*255).astype(np.uint8)

### Load the model
DEEPLAB_MODEL_PATH = '/content/drive/MyDrive/s_data/(extra)/deeplab_model.pb'
with open(DEEPLAB_MODEL_PATH, "rb") as f:
    graph_def = tf.compat.v1.GraphDef.FromString(f.read())
graph = tf.Graph()
with graph.as_default():
    tf.import_graph_def(graph_def=graph_def, name="")
sess = tf.compat.v1.Session(graph=graph)

DEEPLAB_INPUT_SIZE = 513

In [ ]:
### Get the plane model from the road points
def get_road_model_ransac(img_pred,lidar_in_cam,lidar_2d):
  lidar_in_road_lbl = [True if img_pred[pt[1],pt[0]] == 255 else False for pt in lidar_2d]
  lidar_in_road = lidar_in_cam[lidar_in_road_lbl,:]
  road_model = RANSACRegressor().fit(lidar_in_road[:,[0,2]],lidar_in_road[:,1])
  return road_model

### Predict all LiDAR points as road or not
def filter_road_points(road_model,lidar_in,threshold=0.5):
  x = lidar_in[:,[0,2]]
  y_true = lidar_in[:,1]
  y_pred = road_model.predict(x)
  delta_y = np.absolute(y_true-y_pred).flatten()
  is_not_road = delta_y > threshold
  lidar_out = lidar_in[is_not_road,:].copy()
  return lidar_out

road_height_threshold = 0.8

In [ ]:
for v in vehicles:
  print(vehicles[v])
  item=vehicles[v]
  for  i in range(item['num_files']):
    lidar_path=item['lidar_path']
    img_path=item['img_path']
    save_path=item['save_path']
    save_path1=item['save_path1']
    lidar_2_world=np.load(item['lidar_matrices_path']+'/{}.npy'.format(i))
    world_2_camera=np.load(item['camera_matrices_path']+'/{}.npy'.format(i))
    camera_intrinsic=np.load(item['camera_intrinsic_path'])
    if not os.path.exists(save_path):
      os.makedirs(save_path)
    if not os.path.exists(save_path1):
      os.makedirs(save_path1)
    '''if  os.path.exists(save_path+'/{}.npy'.format(i)):
            print("File {} is already created. Skipping item...".format(i))

            continue'''
    if not os.path.exists(lidar_path+'/{}.npy'.format(i)):
            print("File {} not found. Skipping item...".format(i))
            all_files_exist = False
            continue
    lidar_raw=np.load(lidar_path+'/{}.npy'.format(i))
    img_raw=cv2.imread(img_path+'/{}.jpg'.format(i))

    lidar_raw=lidar_raw.reshape(-1, 4)


    lidar_raw = lidar_raw[lidar_raw[:, 2] <= 0, :]
    dist = np.linalg.norm(lidar_raw, axis=1)
    lidar_raw = lidar_raw[dist >= 3]
    intensity = np.array(lidar_raw[:, 3])
    lidar_raw = lidar_raw[:, :3]
    # Define the radius (in meters)
    radius = 2.7

    # Calculate the Euclidean distance between each point in the point cloud and the lidar sensor position
    distances = np.sqrt((lidar_raw[:, 0])**2 + (lidar_raw[:, 1])**2)

    # Filter out all the points that are within the radius of the lidar sensor position
    lidar_raw = lidar_raw[distances > radius, :]


    lidar_raw = transform_coordinate(lidar_raw,lidar_2_world,world_2_camera)

    img_raw_size = img_raw.shape

    lidar_2d,lidar_in_cam ,mask= project_lidar2cam(lidar_raw,camera_intrinsic,img_raw_size)



    # Extract the screen coords (uv) as integers.
    u_coord = lidar_2d[:, 0].astype(int)
    v_coord = lidar_2d[:, 1].astype(int)

    CROP_RH = 1 # Height ratio
    CROP_RW = 2 # Width ratio
    crop_img,lidar_2d,lidar_in_cam = crop_data(img_raw,lidar_2d,lidar_in_cam,CROP_RH,CROP_RW)
    ##Segmenantation using deeplabv3
    segm_prob,segm_pred = process_images(crop_img, sess, DEEPLAB_INPUT_SIZE, 0.4)
    ### Initialize OGM
    ogm_time_0 = np.ones((MAP_SIZE_Y,MAP_SIZE_X)) * 0.5
    road_model = get_road_model_ransac(segm_pred,lidar_in_cam,lidar_2d)
    lidar_nonroad = filter_road_points(road_model,lidar_raw,road_height_threshold)
    np.save(save_path1+'/{}.npy'.format(i),lidar_nonroad)
    print(save_path1+'/{}.npy'.format(i))
    ### Only use the x-z axis of the point (ignore the height axis)
    lidar_ogm = lidar_nonroad[:,[0,1]]


    ogm_step = generate_measurement_ogm(lidar_ogm,ogm_time_0.shape)
    #plt.imshow(((1-ogm_step)*255).astype(np.uint8),cmap='gray')
    #plt.show()
    np.save(save_path+'/{}.npy'.format(i),ogm_step)
    print(save_path+'/{}.npy'.format(i))
    '''### Visualize
    fig,axs = plt.subplots(figsize=(12,6))
    plt.imshow(((1-ogm_step)*255).astype(np.uint8),cmap='gray')
    plt.show()'''

{'lidar_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/Lidar', 'img_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/rgb', 'lidar_matrices_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/pos_matrices/lidar_2_world', 'camera_matrices_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/pos_matrices/world_2_camera', 'camera_intrinsic_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/pos_matrices/K.npy', 'num_files': 25, 'save_path': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/occupancy_grid', 'save_path1': '/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/lidar_nonroad'}


<ipython-input-24-bd5f8f661648>:36: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes,_ = stats.mode(segm_reg.flatten(),axis=None,nan_policy="omit")


/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/lidar_nonroad/0.npy
/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/occupancy_grid/0.npy


<ipython-input-24-bd5f8f661648>:36: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes,_ = stats.mode(segm_reg.flatten(),axis=None,nan_policy="omit")


/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/lidar_nonroad/1.npy
/content/drive/MyDrive/s_data/scenarios/scenario5/Vehicle_2 vehicle.nissan.micra/occupancy_grid/1.npy


...

<ipython-input-24-bd5f8f661648>:36: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes,_ = stats.mode(segm_reg.flatten(),axis=None,nan_policy="omit")


/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/lidar_nonroad/22.npy
/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/occupancy_grid/22.npy


<ipython-input-24-bd5f8f661648>:36: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes,_ = stats.mode(segm_reg.flatten(),axis=None,nan_policy="omit")


/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/lidar_nonroad/23.npy
/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/occupancy_grid/23.npy
/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/lidar_nonroad/24.npy
/content/drive/MyDrive/s_data/scenarios/scenario5/Ego_1 vehicle.audi.a2/occupancy_grid/24.npy


<ipython-input-24-bd5f8f661648>:36: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes,_ = stats.mode(segm_reg.flatten(),axis=None,nan_policy="omit")
